In [1]:
import matplotlib.pyplot as plt
import numpy as np

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    if "google.colab" in sys.modules:
        !pip install -q pyomo
    else:
        !conda install -c conda-forge pyomo
    assert(shutil.which("pyomo"))
    
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk
        except:
            pass
    assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

import pyomo.environ as pe

# Modèle du 2e exemple Pyomo:

maximiser $40 x_1 + 30 x_2$ (profit)

Sujet à $x_1 \leq 40$ (demand)

$x_1 + x_2 \leq 80$ (laborA)

$2x_1 + x_2 \leq 100$ (laborB)

$x_1, x_2 \geq 0$

# Option 1

In [19]:
model = pe.ConcreteModel()

# variables
model.x1 = pe.Var(domain=pe.NonNegativeReals)  # ou within=
model.x2 = pe.Var(domain=pe.NonNegativeReals)

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=40 * model.x1 + 30 * model.x2)

# contraintes
model.demand = pe.Constraint(expr=model.x1 <= 40)
model.laborA = pe.Constraint(expr=model.x1 + model.x2 <= 80)
model.laborB = pe.Constraint(expr=2 * model.x1 + model.x2 <= 100)

# résolution
pe.SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04946017265319824
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [20]:
model.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x1 + 30*x2

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :   x1 :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : x1 + x2 :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : 2*x1 + x2 : 100.0 :   True

6 Declarations: x1 x2 profit demand laborA

In [21]:
model.display()

Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  60.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 2600.0

  Constraints:
    demand : Size=1
        Key  : Lower : Body : Upper
        None :  None : 20.0 :  40.0
    laborA : Size=1
        Key  : Lower : Body : Upper
        None :  None : 80.0 :  80.0
    laborB : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0


In [23]:
model.x1()

20.0

In [7]:
model.x1.value  # idem pour x2

20.0

In [8]:
pe.value(model.x1)

20.0

In [24]:
model.profit()

2600.0

In [10]:
pe.value(model.profit)

2600.0

# Option 2
$x$ avec liste

In [26]:
model = pe.ConcreteModel()

# paramètres
productList = [1, 2]

# variables
model.x = pe.Var(productList, domain=pe.NonNegativeReals)  # ou within=

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=40 * model.x[1] + 30 * model.x[2])

# contraintes
model.demand = pe.Constraint(expr=model.x[1] <= 40)
model.laborA = pe.Constraint(expr=model.x[1] + model.x[2] <= 80)
model.laborB = pe.Constraint(expr=2 * model.x[1] + model.x[2] <= 100)

# résolution
pe.SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07397270202636719
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [27]:
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  20.0 :  None : False : False : NonNegativeReals
          2 :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x[1] + 30*x[2]

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf : x[1] :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :  -Inf : x[1] + x[2] :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 2*x[1] + x[2] : 100.0 :   True

5 Declarations: x profit demand laborA laborB


In [31]:
model.x[1]()

20.0

In [32]:
model.x[1].value

20.0

In [33]:
pe.value(model.x[1])

20.0

# Option 3
objectif et contraintes avec dictionnaires

In [13]:
model = pe.ConcreteModel()

# paramètres
productList = [1, 2]

profit = {1: 40, 
          2: 30}

demand = {1: 40}

labor_total = [80, 100]
labor = {1: [1, 2],
         2: [1, 1]}


# variables
model.x = pe.Var(productList, domain=pe.NonNegativeReals)  # ou within=

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=sum(profit[i] * model.x[i] for i in productList))

# contraintes
model.demand = pe.ConstraintList()
for i in demand:
    model.demand.add(expr=model.x[i] <= demand[i])

model.labor = pe.ConstraintList()
for j in range(len(labor_total)):
    model.labor.add(expr=sum(labor[i][j] * model.x[i] for i in productList) <= labor_total[j])

# résolution
pe.SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.09003710746765137
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [14]:
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  20.0 :  None : False : False : NonNegativeReals
          2 :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x[1] + 30*x[2]

2 Constraint Declarations
    demand : Size=1, Index={1}, Active=True
        Key : Lower : Body : Upper : Active
          1 :  -Inf : x[1] :  40.0 :   True
    labor : Size=2, Index={1, 2}, Active=True
        Key : Lower : Body          : Upper : Active
          1 :  -Inf :   x[1] + x[2] :  80.0 :   True
          2 :  -Inf : 2*x[1] + x[2] : 100.0 :   True

4 Declarations: x profit demand labor


# Option 4
utilisation de *rule*

In [38]:
model = pe.ConcreteModel()

# paramètres
productList = [1, 2]

profit = {1: 40, 
          2: 30}

demand = {1: 40}

labor_total = [80, 100]
labor = {1: [1, 2],
         2: [1, 1]}


# variables
model.x = pe.Var(productList, domain=pe.NonNegativeReals)  # ou within=

# objectif
model.profit = pe.Objective(sense=pe.maximize,
                            expr=sum(profit[i] * model.x[i] for i in productList))

# contraintes
model.demand = pe.ConstraintList()
for i in demand:
    model.demand.add(expr=model.x[i] <= demand[i])

def labor_rule(m, j):
    return sum(labor[i][j] * m.x[i] for i in productList) <= labor_total[j]
model.labor = pe.Constraint([0, 1], rule=labor_rule)    
# model.labor = pe.ConstraintList()
# for j in range(len(labor_total)):
#     model.labor.add(expr=sum(labor[i][j] * model.x[i] for i in productList) <= labor_total[j])

# résolution
pe.SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.036725759506225586
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [16]:
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  20.0 :  None : False : False : NonNegativeReals
          2 :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x[1] + 30*x[2]

2 Constraint Declarations
    demand : Size=1, Index={1}, Active=True
        Key : Lower : Body : Upper : Active
          1 :  -Inf : x[1] :  40.0 :   True
    labor : Size=2, Index={0, 1}, Active=True
        Key : Lower : Body          : Upper : Active
          0 :  -Inf :   x[1] + x[2] :  80.0 :   True
          1 :  -Inf : 2*x[1] + x[2] : 100.0 :   True

4 Declarations: x profit demand labor
